In [ ]:
import numpy as np
from simple_robot import feature_eng, api_get, api_post, get_result
import fastparquet
from feature_engineering import rsi, macd, bollinger_bands, momentum, get_headlines
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
from numpy import array
import datetime
from datetime import datetime

### Carregando dados

In [ ]:
#df_btc = pd.read_parquet('https://drive.google.com/u/0/uc?id=1K1-mXTk426z8ZvbmWrx8zrbC-C6GxxGg&export=download')
#df_doge = pd.read_parquet('https://drive.google.com/u/0/uc?id=17c2r9qbnsxPVxaYukrp6vhTY-CQy8WZa&export=download')
df_eth = pd.read_parquet('https://drive.google.com/u/0/uc?id=1b2nqwR11tWWZ8B1R3AXUB_iCuu1oELm6&export=download')
#df_eth = api_post('cripto_quotation', {'token': token, 'ticker': 'ETHUSDT'})

### Analisando amostra com dados de 2021 somente

In [ ]:
df = df_eth

In [ ]:
df = df.dropna()
df = df[df['datetime'].between('2021-01-01 00:00:00', '2021-12-31 23:59:59')]

In [ ]:
df.head(3)

### Calculando Target

In [ ]:
# Calculando qual a média de close dos próximos 10min
df['forward_average'] = df[::-1]['close'].rolling(10).mean()[::-1].shift(-1)

# Target será a diferença percentual do 'forward_average' com o 'close' atual 
df['target'] = 100*(df['forward_average'] - df['close']) / df['close']

df.head(3)
# Outra possibilidade: target como a diferença entre o proximo minuto e o atual: df['diff']= -df['close'].diff(-1)

### Calculando variaveis independentes

In [ ]:
# Return Feature
df['return'] = round(df['close'] / df['open'] - 1, 3)
# Change Feature
# Change of the price from previous day, absolute value
df['change'] = (df.close - df.close.shift(1)).fillna(0)
# Volatility Feature
df['volatility'] = df.close.ewm(21).std()
# Momentum
df['momentum'] = momentum(df.close, 3)
# RSI (Relative Strength Index)
df['rsi'] = rsi(df)    
# MACD - (Moving Average Convergence/Divergence)
df['macd'], df['signal'] = macd(df)
# Upper Band and Lower Band for Bollinger Bands
df['upper_band'], df['lower_band'] = bollinger_bands(df)
df.dropna(inplace=True)

In [ ]:
# Toda a parte de criação de features está no arquivo simple_robot.py. Aqui apenas chamamos a função. Isso é útil, pois conseguimos usar a mesma função no momento de colocar o robô em produção
df = feature_eng(df)
df.head(3).T

### Salvando o arquivo

In [ ]:
df.to_parquet('data/eth.parquet', compression='gzip')

In [ ]:
#df.to_parquet('data/eth_TUDO.parquet', compression='gzip')